In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/lexihenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
#ds.close()
ds=xr.open_dataset(dir_data+'ar_vars_geomeans_era5_MAX.nc')
ivt_concat_era5_variable=ds['ivt_concat_era5_variable']
iwv_concat_era5_variable=ds['iwv_concat_era5_variable']
s_concat_era5_variable=ds['s_concat_era5_variable']
mfc_concat_era5_variable=ds['mfc_concat_era5_variable']

ivt_concat_era5_fixed=ds['ivt_concat_era5_fixed']
iwv_concat_era5_fixed=ds['iwv_concat_era5_fixed']
s_concat_era5_fixed=ds['s_concat_era5_fixed']
mfc_concat_era5_fixed=ds['mfc_concat_era5_fixed']

ivt_concat_era5_fixed_poleward=ds['ivt_concat_era5_fixed_poleward']
iwv_concat_era5_fixed_poleward=ds['iwv_concat_era5_fixed_poleward']
s_concat_era5_fixed_poleward=ds['s_concat_era5_fixed_poleward']
mfc_concat_era5_fixed_poleward=ds['mfc_concat_era5_fixed_poleward']


ds=xr.open_dataset(dir_data+'ar_vars_geomeans_merra2_MAX.nc')
ivt_concat_merra2_variable=ds['ivt_concat_merra2_variable']
iwv_concat_merra2_variable=ds['iwv_concat_merra2_variable']
s_concat_merra2_variable=ds['s_concat_merra2_variable']
mfc_concat_merra2_variable=ds['mfc_concat_merra2_variable']

ivt_concat_merra2_fixed=ds['ivt_concat_merra2_fixed']
iwv_concat_merra2_fixed=ds['iwv_concat_merra2_fixed']
s_concat_merra2_fixed=ds['s_concat_merra2_fixed']
mfc_concat_merra2_fixed=ds['mfc_concat_merra2_fixed']

ivt_concat_merra2_fixed_poleward=ds['ivt_concat_merra2_fixed_poleward']
iwv_concat_merra2_fixed_poleward=ds['iwv_concat_merra2_fixed_poleward']
s_concat_merra2_fixed_poleward=ds['s_concat_merra2_fixed_poleward']
mfc_concat_merra2_fixed_poleward=ds['mfc_concat_merra2_fixed_poleward']


ds=xr.open_dataset(dir_data+'ar_vars_geomeans_jra55_MAX.nc')
ivt_concat_jra55_variable=ds['ivt_concat_jra55_variable']
iwv_concat_jra55_variable=ds['iwv_concat_jra55_variable']
s_concat_jra55_variable=ds['s_concat_jra55_variable']

ivt_concat_jra55_fixed=ds['ivt_concat_jra55_fixed']
iwv_concat_jra55_fixed=ds['iwv_concat_jra55_fixed']
s_concat_jra55_fixed=ds['s_concat_jra55_fixed']

ivt_concat_jra55_fixed_poleward=ds['ivt_concat_jra55_fixed_poleward']
iwv_concat_jra55_fixed_poleward=ds['iwv_concat_jra55_fixed_poleward']
s_concat_jra55_fixed_poleward=ds['s_concat_jra55_fixed_poleward']

In [ ]:
yrs=np.arange(1980,2024)

lons_west=[x for x in ivt_concat_jra55_fixed.lon.values if x<-90]
lons_east=[x for x in ivt_concat_jra55_fixed.lon.values if x>=90]
sel_west=ivt_concat_jra55_fixed.sel(lon=lons_west)
sel_east=ivt_concat_jra55_fixed.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,ivt_concat_jra55_fixed,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_ivt_fixed=interp.sel(lon=lons_center)


sel_west=ivt_concat_jra55_variable.sel(lon=lons_west)
sel_east=ivt_concat_jra55_variable.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,ivt_concat_jra55_variable,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_ivt_variable=interp.sel(lon=lons_center)


sel_west=ivt_concat_jra55_fixed_poleward.sel(lon=lons_west)
sel_east=ivt_concat_jra55_fixed_poleward.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,ivt_concat_jra55_fixed_poleward,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_ivt_fixed_poleward=interp.sel(lon=lons_center)

####

sel_west=iwv_concat_jra55_fixed.sel(lon=lons_west)
sel_east=iwv_concat_jra55_fixed.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,iwv_concat_jra55_fixed,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_iwv_fixed=interp.sel(lon=lons_center)


sel_west=iwv_concat_jra55_variable.sel(lon=lons_west)
sel_east=iwv_concat_jra55_variable.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,iwv_concat_jra55_variable,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_iwv_variable=interp.sel(lon=lons_center)


sel_west=iwv_concat_jra55_fixed_poleward.sel(lon=lons_west)
sel_east=iwv_concat_jra55_fixed_poleward.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,iwv_concat_jra55_fixed_poleward,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_iwv_fixed_poleward=interp.sel(lon=lons_center)

#####

lons_west=[x for x in s_concat_jra55_fixed.lon1.values if x<-90]
lons_east=[x for x in s_concat_jra55_fixed.lon1.values if x>=90]
sel_west=s_concat_jra55_fixed.sel(lon1=lons_west)
sel_east=s_concat_jra55_fixed.sel(lon1=lons_east)
sel_west['lon1']=[x+360. for x in lons_west]
sel_east['lon1']=[x-360. for x in lons_east]

final=xr.concat([sel_east,s_concat_jra55_fixed,sel_west],dim='lon1')
dk=xr.Dataset()
dk['var']=(('time','lat','lon'),final.values)
dk.coords['time']=yrs
dk.coords['lat']=final.lat1.values
dk.coords['lon']=final.lon1.values
final1=dk['var']

interp=final1.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_s_fixed=interp.sel(lon=lons_center)


sel_west=s_concat_jra55_variable.sel(lon1=lons_west)
sel_east=s_concat_jra55_variable.sel(lon1=lons_east)
sel_west['lon1']=[x+360. for x in lons_west]
sel_east['lon1']=[x-360. for x in lons_east]
final=xr.concat([sel_east,s_concat_jra55_variable,sel_west],dim='lon1')

dk=xr.Dataset()
dk['var']=(('time','lat','lon'),final.values)
dk.coords['time']=yrs
dk.coords['lat']=final.lat1.values
dk.coords['lon']=final.lon1.values
final1=dk['var']

interp=final1.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_s_variable=interp.sel(lon=lons_center)


sel_west=s_concat_jra55_fixed_poleward.sel(lon1=lons_west)
sel_east=s_concat_jra55_fixed_poleward.sel(lon1=lons_east)
sel_west['lon1']=[x+360. for x in lons_west]
sel_east['lon1']=[x-360. for x in lons_east]
final=xr.concat([sel_east,s_concat_jra55_fixed_poleward,sel_west],dim='lon1')

dk=xr.Dataset()
dk['var']=(('time','lat','lon'),final.values)
dk.coords['time']=yrs
dk.coords['lat']=final.lat1.values
dk.coords['lon']=final.lon1.values
final1=dk['var']

interp=final1.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_s_fixed_poleward=interp.sel(lon=lons_center)

In [ ]:
yrs=np.arange(1980,2024,1)
slopes_array_ivt_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

slopes_array_iwv_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

slopes_array_s_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_jra55_variable=xr.zeros_like(s_concat_era5_variable[0,:,:])
slopes_array_s_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_jra55_fixed=xr.zeros_like(s_concat_era5_variable[0,:,:])
slopes_array_s_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_jra55_fixed_poleward=xr.zeros_like(s_concat_era5_variable[0,:,:])

slopes_array_mfc_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#slopes_array_mfc_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#slopes_array_mfc_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#slopes_array_mfc_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])



sigs_array_ivt_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

sigs_array_iwv_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

sigs_array_s_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_jra55_variable=xr.zeros_like(s_concat_era5_variable[0,:,:])
sigs_array_s_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_jra55_fixed=xr.zeros_like(s_concat_era5_variable[0,:,:])
sigs_array_s_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_jra55_fixed_poleward=xr.zeros_like(s_concat_era5_variable[0,:,:])

sigs_array_mfc_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#sigs_array_mfc_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#sigs_array_mfc_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#sigs_array_mfc_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

size=20
for j in range(slopes_array_ivt_era5_variable.lat.size):
    print(j)
    for k in range(slopes_array_ivt_era5_variable.lon.size):
        time_series=ivt_concat_era5_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5_variable[j,k]=r[0]
            sigs_array_ivt_era5_variable[j,k]=r[3]
        else:
            slopes_array_ivt_era5_variable[j,k]=np.nan
            sigs_array_ivt_era5_variable[j,k]=np.nan
        
        time_series=iwv_concat_era5_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5_variable[j,k]=r[0]
            sigs_array_iwv_era5_variable[j,k]=r[3]
        else:
            slopes_array_iwv_era5_variable[j,k]=np.nan
            sigs_array_iwv_era5_variable[j,k]=np.nan
        
        time_series=s_concat_era5_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5_variable[j,k]=r[0]
            sigs_array_s_era5_variable[j,k]=r[3]
        else:
            slopes_array_s_era5_variable[j,k]=np.nan
            sigs_array_s_era5_variable[j,k]=np.nan

        time_series=mfc_concat_era5_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5_variable[j,k]=r[0]
            sigs_array_mfc_era5_variable[j,k]=r[3]
        else:
            slopes_array_mfc_era5_variable[j,k]=np.nan
            sigs_array_mfc_era5_variable[j,k]=np.nan
        
        time_series=ivt_concat_era5_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5_fixed[j,k]=r[0]
            sigs_array_ivt_era5_fixed[j,k]=r[3]
        else:
            slopes_array_ivt_era5_fixed[j,k]=np.nan
            sigs_array_ivt_era5_fixed[j,k]=np.nan
        
        time_series=iwv_concat_era5_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5_fixed[j,k]=r[0]
            sigs_array_iwv_era5_fixed[j,k]=r[3]
        else:
            slopes_array_iwv_era5_fixed[j,k]=np.nan
            sigs_array_iwv_era5_fixed[j,k]=np.nan
        
        time_series=s_concat_era5_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5_fixed[j,k]=r[0]
            sigs_array_s_era5_fixed[j,k]=r[3]
        else:
            slopes_array_s_era5_fixed[j,k]=np.nan
            sigs_array_s_era5_fixed[j,k]=np.nan
        
        time_series=mfc_concat_era5_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5_fixed[j,k]=r[0]
            sigs_array_mfc_era5_fixed[j,k]=r[3]
        else:
            slopes_array_mfc_era5_fixed[j,k]=np.nan
            sigs_array_mfc_era5_fixed[j,k]=np.nan
        
        time_series=ivt_concat_era5_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5_fixed_poleward[j,k]=r[0]
            sigs_array_ivt_era5_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_ivt_era5_fixed_poleward[j,k]=np.nan
            sigs_array_ivt_era5_fixed_poleward[j,k]=np.nan
        
        time_series=iwv_concat_era5_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5_fixed_poleward[j,k]=r[0]
            sigs_array_iwv_era5_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_iwv_era5_fixed_poleward[j,k]=np.nan
            sigs_array_iwv_era5_fixed_poleward[j,k]=np.nan

        time_series=s_concat_era5_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5_fixed_poleward[j,k]=r[0]
            sigs_array_s_era5_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_s_era5_fixed_poleward[j,k]=np.nan
            sigs_array_s_era5_fixed_poleward[j,k]=np.nan
        
        time_series=mfc_concat_era5_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5_fixed_poleward[j,k]=r[0]
            sigs_array_mfc_era5_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_mfc_era5_fixed_poleward[j,k]=np.nan
            sigs_array_mfc_era5_fixed_poleward[j,k]=np.nan
        
        
        #############
        
        time_series=ivt_concat_merra2_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_merra2_variable[j,k]=r[0]
            sigs_array_ivt_merra2_variable[j,k]=r[3]
        else:
            slopes_array_ivt_merra2_variable[j,k]=np.nan
            sigs_array_ivt_merra2_variable[j,k]=np.nan
        
        time_series=iwv_concat_merra2_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_merra2_variable[j,k]=r[0]
            sigs_array_iwv_merra2_variable[j,k]=r[3]
        else:
            slopes_array_iwv_merra2_variable[j,k]=np.nan
            sigs_array_iwv_merra2_variable[j,k]=np.nan
        
        time_series=s_concat_merra2_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_merra2_variable[j,k]=r[0]
            sigs_array_s_merra2_variable[j,k]=r[3]
        else:
            slopes_array_s_merra2_variable[j,k]=np.nan
            sigs_array_s_merra2_variable[j,k]=np.nan
        
        time_series=mfc_concat_merra2_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_merra2_variable[j,k]=r[0]
            sigs_array_mfc_merra2_variable[j,k]=r[3]
        else:
            slopes_array_mfc_merra2_variable[j,k]=np.nan
            sigs_array_mfc_merra2_variable[j,k]=np.nan
        
        time_series=ivt_concat_merra2_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_merra2_fixed[j,k]=r[0]
            sigs_array_ivt_merra2_fixed[j,k]=r[3]
        else:
            slopes_array_ivt_merra2_fixed[j,k]=np.nan
            sigs_array_ivt_merra2_fixed[j,k]=np.nan
        
        time_series=iwv_concat_merra2_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_merra2_fixed[j,k]=r[0]
            sigs_array_iwv_merra2_fixed[j,k]=r[3]
        else:
            slopes_array_iwv_merra2_fixed[j,k]=np.nan
            sigs_array_iwv_merra2_fixed[j,k]=np.nan
        
        time_series=s_concat_merra2_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_merra2_fixed[j,k]=r[0]
            sigs_array_s_merra2_fixed[j,k]=r[3]
        else:
            slopes_array_s_merra2_fixed[j,k]=np.nan
            sigs_array_s_merra2_fixed[j,k]=np.nan
        
        time_series=mfc_concat_merra2_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_merra2_fixed[j,k]=r[0]
            sigs_array_mfc_merra2_fixed[j,k]=r[3]
        else:
            slopes_array_mfc_merra2_fixed[j,k]=np.nan
            sigs_array_mfc_merra2_fixed[j,k]=np.nan
        
        time_series=ivt_concat_merra2_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_merra2_fixed_poleward[j,k]=r[0]
            sigs_array_ivt_merra2_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_ivt_merra2_fixed_poleward[j,k]=np.nan
            sigs_array_ivt_merra2_fixed_poleward[j,k]=np.nan
        
        time_series=iwv_concat_merra2_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_merra2_fixed_poleward[j,k]=r[0]
            sigs_array_iwv_merra2_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_iwv_merra2_fixed_poleward[j,k]=np.nan
            sigs_array_iwv_merra2_fixed_poleward[j,k]=np.nan
        
        time_series=s_concat_merra2_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_merra2_fixed_poleward[j,k]=r[0]
            sigs_array_s_merra2_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_s_merra2_fixed_poleward[j,k]=np.nan
            sigs_array_s_merra2_fixed_poleward[j,k]=np.nan
            
        time_series=mfc_concat_merra2_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_merra2_fixed_poleward[j,k]=r[0]
            sigs_array_mfc_merra2_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_mfc_merra2_fixed_poleward[j,k]=np.nan
            sigs_array_mfc_merra2_fixed_poleward[j,k]=np.nan
        
        
        #############
        
        time_series=fullres_ivt_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_jra55_variable[j,k]=r[0]
            sigs_array_ivt_jra55_variable[j,k]=r[3]
        else:
            slopes_array_ivt_jra55_variable[j,k]=np.nan
            sigs_array_ivt_jra55_variable[j,k]=np.nan
        
        time_series=fullres_iwv_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_jra55_variable[j,k]=r[0]
            sigs_array_iwv_jra55_variable[j,k]=r[3]
        else:
            slopes_array_iwv_jra55_variable[j,k]=np.nan
            sigs_array_iwv_jra55_variable[j,k]=np.nan
        
        time_series=fullres_ivt_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_jra55_fixed[j,k]=r[0]
            sigs_array_ivt_jra55_fixed[j,k]=r[3]
        else:
            slopes_array_ivt_jra55_fixed[j,k]=np.nan
            sigs_array_ivt_jra55_fixed[j,k]=np.nan
        
        time_series=fullres_iwv_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_jra55_fixed[j,k]=r[0]
            sigs_array_iwv_jra55_fixed[j,k]=r[3]
        else:
            slopes_array_iwv_jra55_fixed[j,k]=np.nan
            sigs_array_iwv_jra55_fixed[j,k]=np.nan
            
        time_series=fullres_ivt_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_jra55_fixed_poleward[j,k]=r[0]
            sigs_array_ivt_jra55_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_ivt_jra55_fixed_poleward[j,k]=np.nan
            sigs_array_ivt_jra55_fixed_poleward[j,k]=np.nan
        
        time_series=fullres_iwv_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_jra55_fixed_poleward[j,k]=r[0]
            sigs_array_iwv_jra55_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_iwv_jra55_fixed_poleward[j,k]=np.nan
            sigs_array_iwv_jra55_fixed_poleward[j,k]=np.nan
        
        time_series=fullres_s_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_jra55_variable[j,k]=r[0]
            sigs_array_s_jra55_variable[j,k]=r[3]
        else:
            slopes_array_s_jra55_variable[j,k]=np.nan
            sigs_array_s_jra55_variable[j,k]=np.nan
        
        time_series=fullres_s_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_jra55_fixed[j,k]=r[0]
            sigs_array_s_jra55_fixed[j,k]=r[3]
        else:
            slopes_array_s_jra55_fixed[j,k]=np.nan
            sigs_array_s_jra55_fixed[j,k]=np.nan
        
        time_series=fullres_s_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_jra55_fixed_poleward[j,k]=r[0]
            sigs_array_s_jra55_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_s_jra55_fixed_poleward[j,k]=np.nan
            sigs_array_s_jra55_fixed_poleward[j,k]=np.nan

In [ ]:
dk=xr.Dataset()
dk['slopes_array_ivt_era5_variable']=(('lat','lon'),slopes_array_ivt_era5_variable.values)
dk['slopes_array_ivt_era5_fixed']=(('lat','lon'),slopes_array_ivt_era5_fixed.values)
dk['slopes_array_ivt_era5_fixed_poleward']=(('lat','lon'),slopes_array_ivt_era5_fixed_poleward.values)
dk['slopes_array_ivt_merra2_variable']=(('lat','lon'),slopes_array_ivt_merra2_variable.values)
dk['slopes_array_ivt_merra2_fixed']=(('lat','lon'),slopes_array_ivt_merra2_fixed.values)
dk['slopes_array_ivt_merra2_fixed_poleward']=(('lat','lon'),slopes_array_ivt_merra2_fixed_poleward.values)
dk['slopes_array_ivt_jra55_variable']=(('lat','lon'),slopes_array_ivt_jra55_variable.values)
dk['slopes_array_ivt_jra55_fixed']=(('lat','lon'),slopes_array_ivt_jra55_fixed.values)
dk['slopes_array_ivt_jra55_fixed_poleward']=(('lat','lon'),slopes_array_ivt_jra55_fixed_poleward.values)

dk['sigs_array_ivt_era5_variable']=(('lat','lon'),sigs_array_ivt_era5_variable.values)
dk['sigs_array_ivt_era5_fixed']=(('lat','lon'),sigs_array_ivt_era5_fixed.values)
dk['sigs_array_ivt_era5_fixed_poleward']=(('lat','lon'),sigs_array_ivt_era5_fixed_poleward.values)
dk['sigs_array_ivt_merra2_variable']=(('lat','lon'),sigs_array_ivt_merra2_variable.values)
dk['sigs_array_ivt_merra2_fixed']=(('lat','lon'),sigs_array_ivt_merra2_fixed.values)
dk['sigs_array_ivt_merra2_fixed_poleward']=(('lat','lon'),sigs_array_ivt_merra2_fixed_poleward.values)
dk['sigs_array_ivt_jra55_variable']=(('lat','lon'),sigs_array_ivt_jra55_variable.values)
dk['sigs_array_ivt_jra55_fixed']=(('lat','lon'),sigs_array_ivt_jra55_fixed.values)
dk['sigs_array_ivt_jra55_fixed_poleward']=(('lat','lon'),sigs_array_ivt_jra55_fixed_poleward.values)



dk['slopes_array_iwv_era5_variable']=(('lat','lon'),slopes_array_iwv_era5_variable.values)
dk['slopes_array_iwv_era5_fixed']=(('lat','lon'),slopes_array_iwv_era5_fixed.values)
dk['slopes_array_iwv_era5_fixed_poleward']=(('lat','lon'),slopes_array_iwv_era5_fixed_poleward.values)
dk['slopes_array_iwv_merra2_variable']=(('lat','lon'),slopes_array_iwv_merra2_variable.values)
dk['slopes_array_iwv_merra2_fixed']=(('lat','lon'),slopes_array_iwv_merra2_fixed.values)
dk['slopes_array_iwv_merra2_fixed_poleward']=(('lat','lon'),slopes_array_iwv_merra2_fixed_poleward.values)
dk['slopes_array_iwv_jra55_variable']=(('lat','lon'),slopes_array_iwv_jra55_variable.values)
dk['slopes_array_iwv_jra55_fixed']=(('lat','lon'),slopes_array_iwv_jra55_fixed.values)
dk['slopes_array_iwv_jra55_fixed_poleward']=(('lat','lon'),slopes_array_iwv_jra55_fixed_poleward.values)

dk['sigs_array_iwv_era5_variable']=(('lat','lon'),sigs_array_iwv_era5_variable.values)
dk['sigs_array_iwv_era5_fixed']=(('lat','lon'),sigs_array_iwv_era5_fixed.values)
dk['sigs_array_iwv_era5_fixed_poleward']=(('lat','lon'),sigs_array_iwv_era5_fixed_poleward.values)
dk['sigs_array_iwv_merra2_variable']=(('lat','lon'),sigs_array_iwv_merra2_variable.values)
dk['sigs_array_iwv_merra2_fixed']=(('lat','lon'),sigs_array_iwv_merra2_fixed.values)
dk['sigs_array_iwv_merra2_fixed_poleward']=(('lat','lon'),sigs_array_iwv_merra2_fixed_poleward.values)
dk['sigs_array_iwv_jra55_variable']=(('lat','lon'),sigs_array_iwv_jra55_variable.values)
dk['sigs_array_iwv_jra55_fixed']=(('lat','lon'),sigs_array_iwv_jra55_fixed.values)
dk['sigs_array_iwv_jra55_fixed_poleward']=(('lat','lon'),sigs_array_iwv_jra55_fixed_poleward.values)



dk['slopes_array_s_era5_variable']=(('lat','lon'),slopes_array_s_era5_variable.values)
dk['slopes_array_s_era5_fixed']=(('lat','lon'),slopes_array_s_era5_fixed.values)
dk['slopes_array_s_era5_fixed_poleward']=(('lat','lon'),slopes_array_s_era5_fixed_poleward.values)
dk['slopes_array_s_merra2_variable']=(('lat','lon'),slopes_array_s_merra2_variable.values)
dk['slopes_array_s_merra2_fixed']=(('lat','lon'),slopes_array_s_merra2_fixed.values)
dk['slopes_array_s_merra2_fixed_poleward']=(('lat','lon'),slopes_array_s_merra2_fixed_poleward.values)
dk['slopes_array_s_jra55_variable']=(('lat','lon'),slopes_array_s_jra55_variable.values)
dk['slopes_array_s_jra55_fixed']=(('lat','lon'),slopes_array_s_jra55_fixed.values)
dk['slopes_array_s_jra55_fixed_poleward']=(('lat','lon'),slopes_array_s_jra55_fixed_poleward.values)

dk['sigs_array_s_era5_variable']=(('lat','lon'),sigs_array_s_era5_variable.values)
dk['sigs_array_s_era5_fixed']=(('lat','lon'),sigs_array_s_era5_fixed.values)
dk['sigs_array_s_era5_fixed_poleward']=(('lat','lon'),sigs_array_s_era5_fixed_poleward.values)
dk['sigs_array_s_merra2_variable']=(('lat','lon'),sigs_array_s_merra2_variable.values)
dk['sigs_array_s_merra2_fixed']=(('lat','lon'),sigs_array_s_merra2_fixed.values)
dk['sigs_array_s_merra2_fixed_poleward']=(('lat','lon'),sigs_array_s_merra2_fixed_poleward.values)
dk['sigs_array_s_jra55_variable']=(('lat','lon'),sigs_array_s_jra55_variable.values)
dk['sigs_array_s_jra55_fixed']=(('lat','lon'),sigs_array_s_jra55_fixed.values)
dk['sigs_array_s_jra55_fixed_poleward']=(('lat','lon'),sigs_array_s_jra55_fixed_poleward.values)



dk['slopes_array_mfc_era5_variable']=(('lat','lon'),slopes_array_mfc_era5_variable.values)
dk['slopes_array_mfc_era5_fixed']=(('lat','lon'),slopes_array_mfc_era5_fixed.values)
dk['slopes_array_mfc_era5_fixed_poleward']=(('lat','lon'),slopes_array_mfc_era5_fixed_poleward.values)
dk['slopes_array_mfc_merra2_variable']=(('lat','lon'),slopes_array_mfc_merra2_variable.values)
dk['slopes_array_mfc_merra2_fixed']=(('lat','lon'),slopes_array_mfc_merra2_fixed.values)
dk['slopes_array_mfc_merra2_fixed_poleward']=(('lat','lon'),slopes_array_mfc_merra2_fixed_poleward.values)

dk['sigs_array_mfc_era5_variable']=(('lat','lon'),sigs_array_mfc_era5_variable.values)
dk['sigs_array_mfc_era5_fixed']=(('lat','lon'),sigs_array_mfc_era5_fixed.values)
dk['sigs_array_mfc_era5_fixed_poleward']=(('lat','lon'),sigs_array_mfc_era5_fixed_poleward.values)
dk['sigs_array_mfc_merra2_variable']=(('lat','lon'),sigs_array_mfc_merra2_variable.values)
dk['sigs_array_mfc_merra2_fixed']=(('lat','lon'),sigs_array_mfc_merra2_fixed.values)
dk['sigs_array_mfc_merra2_fixed_poleward']=(('lat','lon'),sigs_array_mfc_merra2_fixed_poleward.values)




dk.coords['lat']=slopes_array_ivt_era5_variable.lat.values
dk.coords['lon']=slopes_array_ivt_era5_variable.lon.values

dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'

try:
    os.remove(dir_data+'synoptic_geotrends_arrays_variants_MAX.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'synoptic_geotrends_arrays_variants_MAX.nc',mode='w',format='NETCDF4')